# Telegram-бот, детоксифицирующий сообщения пользователей
https://t.me/detoxify_bot

In [1]:
import telebot
from telebot import types

import json

In [2]:
with open('./config.json', 'r') as f:
    json_config = json.load(f)

In [3]:
TOKEN = json_config['tg_bot_token']

bot = telebot.TeleBot(TOKEN, num_threads=4)

In [4]:
from transformers import AutoTokenizer, T5ForConditionalGeneration

tokenizer = AutoTokenizer.from_pretrained("ai-forever/ruT5-base")
model = T5ForConditionalGeneration.from_pretrained("../best_models/model_ruT5_5e-5_withPreprocessing_V1")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [5]:
def paraphrase(text, model, n=None, max_length='auto', temperature=0.0, beams=3):
    texts = [text] if isinstance(text, str) else text
    inputs = tokenizer(texts, return_tensors='pt', padding=True)['input_ids'].to(model.device)
    if max_length == 'auto':
        max_length = int(inputs.shape[1] * 1.2) + 10
    result = model.generate(
        inputs,
        num_return_sequences=n or 1,
        do_sample=False,
        temperature=temperature,
        repetition_penalty=3.0,
        max_length=max_length,
        bad_words_ids=[[2]],  # unk
        num_beams=beams,
    )
    texts = [tokenizer.decode(r, skip_special_tokens=True) for r in result]
    if not n and isinstance(text, str):
        return texts[0]
    return texts

In [6]:
# Handle '/start' and '/help'
@bot.message_handler(commands=['start'])
def send_welcome(message):
    welcome_message = """Привет! Я бот-детоксификатор. Напиши мне какое-нибудь сообщение, и я попытаюсь его детоксифицировать."""
    bot.reply_to(message, welcome_message)

In [7]:
@bot.message_handler(commands=['help'])
def send_welcome(message):
    help_message = """В основе лежит модель ruT5-base (https://huggingface.co/ai-forever/ruT5-base), которую зафайнтьюнили для задачи детоксификации текстов на русском языке."""
    bot.reply_to(message, help_message)

In [8]:
@bot.message_handler(func=lambda message: True)
def paraphrase_message(message):
    detoxified = paraphrase(message.text, model)
    bot.reply_to(message, detoxified)

In [9]:
bot.infinity_polling()

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
2023-09-09 19:33:36,376 (__init__.py:966 MainThread) ERROR - TeleBot: "Infinity polling: polling exited"
2023-09-09 19:33:36,378 (__init__.py:968 MainThread) ERROR - TeleBot: "Break infinity polling"
